In [1]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz

!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"
# Start a SparkSession
import findspark
findspark.init()
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigDataHW").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()


The system cannot find the path specified.
'wget' is not recognized as an internal or external command,
operable program or batch file.
tar: Error opening archive: Failed to open 'spark-2.4.5-bin-hadoop2.7.tgz'


IndexError: list index out of range

In [0]:
# Configure settings for RDS
#mode = "append"
jdbc_url="jdbc:postgresql://accident-viz.c4cdhyeva5ut.us-east-1.rds.amazonaws.com:5432/Accident-ETL"
config = {"user":"postgres",
          "password": "traffic2020",
          "driver":"org.postgresql.Driver"}

#spark.sparkContext.addFile(jdbc_url)
#beauty_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Beauty_v1_00.tsv.gz"), sep="\t", header=True)


In [0]:
# Get the id, age where age = 22 in SQL

#sqlquery = "select name,salary from testdb.employee"
#df_select = spark.read.jdbc(url=jdbc_url,table=_sqlquery,properties=db_properties=config)

#sqlquery = "(SELECT * FROM accidents where temperature>'90.0' as accdnts)"
sql_query = "(SELECT * FROM accidents) as accdnts"

spark_df = spark.read.jdbc(url=jdbc_url, table = sql_query, column = 'severity', lowerBound =  1, upperBound = 4, numPartitions = 4, properties=config)

#spark_df = spark.read.jdbc(url=jdbc_url, table = sqlquery, properties=config)

#spark_df = spark.sql("SELECT * FROM accidents")


In [0]:
######## Convert Spark DF to Pandas DF
import numpy as np
import pandas as pd
# Enable Arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.enabled", "true")


In [0]:
# Convert the Spark DataFrame to a pandas DataFrame using Arrow
pandas_df = pd.DataFrame()
#result_pdf = spark_df.toPandas()

In [0]:
n = spark_df.count()

In [0]:
n = spark_df.count()
chunksize = 80000
for i in range (0, n, chunksize):
  chunk = spark_df.filter(spark_df["rowid"].between(i, i + chunksize))
  pd_df = chunk.toPandas()
  pandas_df= pandas_df.append(pd_df)

In [14]:
# Check the size of the converted DataFrame
len(pandas_df)




2494249

In [15]:
spark_df.show()

+-----------+--------+-------------------+-------------------+------------------+-------------------+--------------------+--------+----+-----------+--------+--------+----------+--------------+----------+-------------+-----------------+-------+--------+--------+-------+-------+-----+--------------+--------------+-------+
|accident_id|severity|         start_time|           end_time|         start_lat|          start_lng|         coordinates|distance|side|temperature|humidity|pressure|visibility|wind_direction|wind_speed|precipitation|weather_condition|amenity|crossing|junction|railway|station| stop|traffic_signal|civil_twilight|  rowid|
+-----------+--------+-------------------+-------------------+------------------+-------------------+--------------------+--------+----+-----------+--------+--------+----------+--------------+----------+-------------+-----------------+-------+--------+--------+-------+-------+-----+--------------+--------------+-------+
|   A-332457|       1|2017-01-30 1

In [16]:
# Determine the size of the dataframe
print(len(spark_df.columns), spark_df.count())

26 2494218


In [0]:
# read in accident_location table
sqlquery_location = "(SELECT * FROM accident_location where state='CA') as locations"
spark_loc_df = spark.read.jdbc(url=jdbc_url, table = sqlquery_location, properties=config)


In [18]:
spark_loc_df.show()

+--------------------+-------------------+---------------+------------+-----+-------+----------+-------+
|         coordinates|             street|           city|      county|state|zipcode|  timezone|highway|
+--------------------+-------------------+---------------+------------+-----+-------+----------+-------+
|36.73457:-120.199785| W Whitesbridge Ave|         Kerman|      Fresno|   CA|  93630|US/Pacific|      N|
| 37.31641:-121.95618|            I-280 S|       San Jose| Santa Clara|   CA|  95117|US/Pacific|      Y|
|36.73455:-120.205...| W Whitesbridge Ave|   Tranquillity|      Fresno|   CA|  93668|US/Pacific|      N|
| 37.31469:-121.94078|            CA-17 S|       San Jose| Santa Clara|   CA|  95128|US/Pacific|      Y|
|37.31485999999999...|     Sinclair Fwy N|       San Jose| Santa Clara|   CA|  95126|US/Pacific|      Y|
|37.3172:-121.9320...|       S Bascom Ave|       San Jose| Santa Clara|   CA|  95128|US/Pacific|      N|
|37.87589000000000...|      Camino Diablo|      Brentwo

In [19]:
# Determine the size of the dataframe
print(len(spark_loc_df.columns), spark_loc_df.count())

8 201190


Create first frame to read in all accidents with severity = 4
Create second frame to read in all accident locations = CA
Create union DF between the two frames

In [20]:
#DataFrame.join(self, other, on=None, how='left', lsuffix='', rsuffix='', sort=False)
accident_join = spark_df.join(spark_loc_df, on='coordinates', how='inner')#    , lsuffix='*acc', rsuffix="*acc_loc", sort='False')
accident_join.show()


Py4JJavaError: ignored

In [0]:
# Determine the size of the dataframe
print(len(accident_join.columns), accident_join.count())

# Determine which columns to drop, add and which to create dummies
### separate start_time/end_time into components (date & time)
### net end_time from start_time (ensure that start_date = end_date in case accident occurs near midnight)
### create dummies for side (R & L) or reassign to 0, 1
### create dummies for wind_direction
### create dummies for weather_condition
### create 0,1 for amenity, crossing, junction, railway, station, stop, traffic_signal and possibly civil_twilight
### create dummies for timezone
### create 0,1 for highway
### Scale inputs

In [0]:
accident_join.dtypes

In [0]:
# Copy accident_join
#X_df = accident_join.select('*').toPandas()
#X_df = accident_join.toPandas()
X_df = accident_join



In [0]:
# Drop columns

columns_to_drop = ['coordinates', 'accident_ID', 'street', 'city', 'county']
X_df = spark_df.drop(*columns_to_drop)


In [22]:
X_df.dtypes

#X_df.filter(X_df("distance")>0.5).show()

[('severity', 'int'),
 ('start_time', 'timestamp'),
 ('end_time', 'timestamp'),
 ('start_lat', 'double'),
 ('start_lng', 'double'),
 ('distance', 'float'),
 ('side', 'string'),
 ('temperature', 'float'),
 ('humidity', 'float'),
 ('pressure', 'float'),
 ('visibility', 'float'),
 ('wind_direction', 'string'),
 ('wind_speed', 'string'),
 ('precipitation', 'string'),
 ('weather_condition', 'string'),
 ('amenity', 'string'),
 ('crossing', 'string'),
 ('junction', 'string'),
 ('railway', 'string'),
 ('station', 'string'),
 ('stop', 'string'),
 ('traffic_signal', 'string'),
 ('civil_twilight', 'string'),
 ('rowid', 'int')]

In [0]:
X_dummies_df = pd.get_dummies(X_df, columns=["amenity"])

#X_dummies_df = pd.get_dummies(X_df, columns=["side", "wind_direction", "weather_condition", "amenity", "crossing", "junction", "railway", "station", "stop", "traffic_signal", "civil_twilight", "timezone", "highway"])



In [26]:
X_dummies_df.head()

,"DataFrame[severity: int, start_time: timestamp, end_time: timestamp, start_lat: double, start_lng: double, distance: float, side: string, temperature: float, humidity: float, pressure: float, visibility: float, wind_direction: string, wind_speed: string, precipitation: string, weather_condition: string, amenity: string, crossing: string, junction: string, railway: string, station: string, stop: string, traffic_signal: string, civil_twilight: string, rowid: int]"
0,1


In [23]:
X_dummies_df = (X_df).select("*").toPandas()
X_dummies_df = X_df.copy()



/content/spark-2.4.5-bin-hadoop2.7/python/pyspark/sql/dataframe.py:2146: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true, but has reached the error below and can not continue. Note that 'spark.sql.execution.arrow.fallback.enabled' does not have an effect on failures in the middle of computation.
  An error occurred while calling o595.getResult.
: org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:226)
	at org.apache.spark.api.python.PythonServer.getResult(PythonRDD.scala:874)
	at org.apache.spark.api.python.PythonServer.getResult(PythonRDD.scala:870)
	at sun.reflect.GeneratedMethodAccessor62.invoke(Unknown Source)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.Ref

Py4JJavaError: ignored

In [0]:
X_dummies_df = pd.get_dummies(X_dummies_df, columns=["amenity"])

In [0]:
X_dummies_df

In [0]:
#X_dummies_df.empty
#X_dummies_df.columns
X_df['side']

In [0]:
#X_df = pd.get_dummies(X_df, columns=["amenity"])
X_df.head()


#X_df['amenity'] = X_df['amenity'].astype(object)#, columns=['amenity'])

In [0]:
accident_join.show()

In [0]:
y = accident_join['severity']
y.show()

In [0]:
X_ps_df = SparkSession.createDataFrame(X_df).collect()